<a href="https://colab.research.google.com/github/AnkitRajSri/SentimentDetectorUsingChatGPT/blob/main/SentimentDetectorUsingChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the dependencies

In [1]:
!pip install openai

In [39]:
!pip install configobj

In [46]:
!pip install newsapi-python

In [47]:
import os
import openai
import requests
import json
import pandas as pd
from datetime import datetime
from configobj import ConfigObj
from newsapi import NewsApiClient

### Read the config arguments

In [41]:
config_obj = ConfigObj("config_gpt_sa.ini")

In [42]:
openai_api_key = config_obj["OPENAI"]["openai_api_key"]
ninjas_api_key = config_obj["APININJAS"]["ninjas_api_key"]
news_api_key = config_obj["NEWSAPI"]["news_api_key"]

In [43]:
openai.api_key = openai_api_key

### App functions

##### Sentiment detector function

In [45]:
def gpt_sentiment_detector(prompt, emotions):
  sys_prompt = f"""
  You are an emotionally intelligent assistant.
  Classify the sentiment of the user"s text with ONLY ONE OF THE FOLLOWING EMOTIONS: {emotions}.
  After classifying the text, respond with the emotion ONLY.
  """

  messages = [
      {"role": "system",
       "content": sys_prompt},
      {"role": "user",
       "content": prompt}
  ]

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      max_tokens=20,
      temperature=0
  )

  r = response.choices[0].message.content
  if r == "":
    return "N/A"
  return r

### Function to fetch quotes from apininjas

In [27]:
def fetch_quotes(category):
  api_url = "https://api.api-ninjas.com/v1/quotes?category={}".format(category)
  response = requests.get(api_url, headers={"X-Api-Key": ninjas_api_key})
  if response.status_code == requests.codes.ok:
      return response.json()[0]["quote"]
  else:
      print("Error:", response.status_code, response.text)

##### Fetch news using newsapi

In [57]:
news_api = NewsApiClient(api_key=news_api_key)

In [107]:
def get_articles(query, sources, language="en", page_size=1):
  top_articles = news_api.get_everything(q=query,
                                         sources=sources,
                                         language=language,
                                         page_size=page_size)
  return top_articles["articles"][0]["title"]

### Test the sentiment detector

##### Test Case 1: Detect the emotion of a quote

In [124]:
emotions = "happy, funny, sad, angry, mad, tired, very happy, very sad, very angry, very tired, very mad"

In [34]:
quote = fetch_quotes("funny")
print(quote)

Mmmm... the comedy that matters is the comedy you pull out of thin air. It's a bit like when something funny has happened and you try to explain it to someone else and end up saying, 'You had to be there.'


In [37]:
print(gpt_sentiment_detector(quote, emotions))

funny


##### Test Case 2: Detect the tone of a news article

In [111]:
emotions = "Formal, Informal, Optimistic, Worried, Friendly, Curious, Assertive, Encouraging"

In [113]:
article = get_articles("Rise of Artificial Intelligence", "google-news")
print(article)

Artificial intelligence: Why India risks falling behind in the tech race - Business Standard


In [114]:
print(gpt_sentiment_detector(article, emotions))

Worried


In [119]:
article = get_articles("IPL 2023", "google-news")
print(article)

India's fantasy sports' revenue rises 24% during IPL 2023 - Redseer ... - Reuters


In [120]:
print(gpt_sentiment_detector(article, emotions))

Informal
